# Import Required Packages

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.nn.modules import MSELoss, L1Loss, BCELoss

import glob
import csv
import cv2
from numpy import array, asarray, ndarray, swapaxes
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
#!pip install torchvision
#!pip install opencv-python

# Download Files

In [3]:
#training controls
batch_size = 64
epochs = 10
training_size = 0.7
learning_rate = 0.001

# input image dimensions
img_rows, img_cols = 268, 182

In [4]:
# data holders
x_test = []
x_train = []
y_test= []
y_train= []
tempY = []

In [5]:
# opening the dataset
dataset = csv.reader(open("MovieGenre415.csv",encoding="utf8",errors='replace'), delimiter=",")

# skipping the header line
next(dataset)

['imdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster']

In [6]:
# extract images from zip folder

import zipfile as zf

files = zf.ZipFile("FinalProject-20230415T194533Z-001.zip", 'r')
files.extractall()
files.close()

In [7]:
# list of image files in SampleMoviePosters folder
flist=glob.glob('FinalProject/*.jpg')  

In [8]:
len(flist)

5347

In [9]:
image_ids = []

for path in flist:
    start = path.rfind("/")+1
    end = len(path)-4
    image_ids.append(path[start:end])
    
#image_ids

In [10]:
import pandas as pd

dataset2 = pd.read_csv("MovieGenre415.csv")
dataset2

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
1,112281,http://www.imdb.com/title/tt112281,Ace Ventura: When Nature Calls (1995),6.3,Adventure|Comedy|Crime,https://images-na.ssl-images-amazon.com/images...
2,114168,http://www.imdb.com/title/tt114168,Powder (1995),6.5,Drama|Fantasy|Mystery,https://images-na.ssl-images-amazon.com/images...
3,114011,http://www.imdb.com/title/tt114011,Now and Then (1995),6.8,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,114117,http://www.imdb.com/title/tt114117,Persuasion,7.7,Drama,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...
5342,313579,http://www.imdb.com/title/tt313579,Seung joi ngo sam (2001),6.6,Romance,https://images-na.ssl-images-amazon.com/images...
5343,4361050,http://www.imdb.com/title/tt4361050,Ouija: Origin of Evil (2016),6.1,Horror|Thriller,https://images-na.ssl-images-amazon.com/images...
5344,1337139,http://www.imdb.com/title/tt1337139,45365 (2009),7.3,Documentary,https://images-na.ssl-images-amazon.com/images...
5345,47495,http://www.imdb.com/title/tt47495,Silver Lode (1954),7.0,Western,https://images-na.ssl-images-amazon.com/images...


# Data Preprocessing

In [11]:
y = []
indexlist = []
classes = tuple()
ids = dataset2.imdbId.values.tolist()
for image_id in image_ids:
    #print(dataset2["imdbId"])
    genres = tuple((dataset2[dataset2["imdbId"] == int(image_id)]["Genre"].values[0]).split("|"))
    if int(image_id) in ids:
        indexlist.append(image_id)
    y.append(genres)
    classes = classes + genres
mlb = MultiLabelBinarizer()
mlb.fit(y)
y = mlb.transform(y)
classes = set(classes)
classes = list(classes)
classes.sort()

In [12]:
y_df = pd.DataFrame(y, columns = classes, index = indexlist)
y_df

,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,Mystery,News,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
408839,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
240468,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
387131,0,0,0,0,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
116409,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
106317,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3331846,0,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
108160,0,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
2247566,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
43386,0,0,0,0,0,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0


In [13]:
y_df_reset = y_df.reset_index()

shape = y_df_reset.shape[1]

index_value = []
genre_lst = []

for i in range(len(y_df_reset)):
    index_value.append(int(y_df_reset.loc[i,"index"]))
    temp_list = []
    for j in y_df_reset.columns[1:]:
        temp_list.append(y_df_reset.loc[i,j])
    genre_lst.append(temp_list)

df = pd.DataFrame(list(zip(index_value, genre_lst)),
               columns =['imdbId', 'genrelst'])

result = dataset2.merge(df, on="imdbId")
result

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster,genrelst
0,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,112281,http://www.imdb.com/title/tt112281,Ace Ventura: When Nature Calls (1995),6.3,Adventure|Comedy|Crime,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,114168,http://www.imdb.com/title/tt114168,Powder (1995),6.5,Drama|Fantasy|Mystery,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ..."
3,114011,http://www.imdb.com/title/tt114011,Now and Then (1995),6.8,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,114117,http://www.imdb.com/title/tt114117,Persuasion,7.7,Drama,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
5342,313579,http://www.imdb.com/title/tt313579,Seung joi ngo sam (2001),6.6,Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5343,4361050,http://www.imdb.com/title/tt4361050,Ouija: Origin of Evil (2016),6.1,Horror|Thriller,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
5344,1337139,http://www.imdb.com/title/tt1337139,45365 (2009),7.3,Documentary,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
5345,47495,http://www.imdb.com/title/tt47495,Silver Lode (1954),7.0,Western,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [14]:
for x in range(len(result)):
    tempY.append((int(result['imdbId'].iloc[x]),result['genrelst'].iloc[x]))

#tempY

# Train/Test Split

In [15]:
#setting the length of training data
length=int(len(flist)*training_size)
length

3742

In [16]:
#extracting the data about the images that are available
i=0
for filename in flist:
    name=int(filename.split('/')[-1][:-4])
    for z in tempY:
        if(z[0]==name):
            
            img = array(cv2.imread(filename))
            img = swapaxes(img, 2,0)
            img = swapaxes(img, 2,1)

            if(i<length):
                x_train.append(img)
                y_train.append(z[1])
                i+=1
            else:
                x_test.append(img)
                y_test.append(z[1])
                i+=1

In [17]:
#converting the data from lists to numpy arrays
x_train=asarray(x_train,dtype=float)
x_test=asarray(x_test,dtype=float)
y_train=asarray(y_train,dtype=float)
y_test=asarray(y_test,dtype=float)

In [18]:
#scaling down the RGB data
x_train /= 255
x_test /= 255

In [19]:
#printing stats about the features
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (3742, 3, 268, 182)
3742 train samples
1605 test samples


In [20]:
train_length = x_train.shape[0]

x_train=torch.from_numpy(x_train)
x_test=torch.from_numpy(x_test)
y_train=torch.from_numpy(y_train)
y_test=torch.from_numpy(y_test)

train = data_utils.TensorDataset(x_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

test = data_utils.TensorDataset(x_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False)

In [21]:
# Metric calculation

def metric(scores, targets):
    """
    :param scores: the output the model predict
    :param targets: the gt label
    :return: OP, OR, OF1, CP, CR, CF1
    calculate the Precision of every class by: TP/TP+FP i.e. TP/total predict
    calculate the Recall by: TP/total GT
    """
    num, num_class = scores.shape
    gt_num = np.zeros(num_class)
    tp_num = np.zeros(num_class)
    predict_num = np.zeros(num_class)


    for index in range(num_class):
        score = scores[:, index]
        target = targets[:, index]

        gt_num[index] = np.sum(target == 1)
        predict_num[index] = np.sum(score >= 0.5)
        tp_num[index] = np.sum(target * (score >= 0.5))

    predict_num[predict_num == 0] = 1  # avoid dividing 0
    OP = np.sum(tp_num) / np.sum(predict_num) #OP (Overall Precision) is the ratio of the number of correctly predicted positive samples to the total number of positive predictions made by the model
    OR = np.sum(tp_num) / np.sum(gt_num) #OR (Overall Recall) is the ratio of the number of correctly predicted positive samples to the total number of positive samples in the ground truth.
    OF1 = (2 * OP * OR) / (OP + OR) #OF1 (Overall F1 Score) is the harmonic mean of precision and recall.

    return OP, OR, OF1

# Model 1: Resnet50

In [22]:
# Resnet50 model
from torchvision import models

class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(2048, len(classes))

    def forward(self, x):
        x = self.resnet(x)
        return x

In [23]:
model = ResNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
from statistics import mean
import numpy as np

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model(data)

        preds = torch.round(output)
            
        #acc_list = []
        #preds = torch.round(output)
        #for i in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for j in range(len(classes)):
        #        if target[i][j] == 1 or preds[i][j] == 1:
        #            denom += 1
        #            if preds[i][j] == target[i][j]:
        #                result+=1
        #    acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model(data)
        
        preds = torch.round(output)
        
        #acc_list = []
        #preds = torch.round(output)
        #for n in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for m in range(len(classes)):
        #        if target[n][m] == 1 or preds[n][m] == 1:
        #            denom += 1
        #            if preds[n][m] == target[n][m]:
        #                result+=1
        #    acc_list.append(result/denom)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP, OR, OF1))

for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/3742 (0%)]	Loss: 0.697382 	OP: 0.111111	OR: 0.066176	OF1: 0.082949
Train Epoch: 0 [64/3742 (2%)]	Loss: 0.658392 	OP: 0.022727	OR: 0.007576	OF1: 0.011364
Train Epoch: 0 [128/3742 (3%)]	Loss: 0.619709 	OP: 0.031250	OR: 0.006897	OF1: 0.011299
Train Epoch: 0 [192/3742 (5%)]	Loss: 0.582666 	OP: 0.035714	OR: 0.006757	OF1: 0.011364


<ipython-input-21-447663f93c68>:28: RuntimeWarning: invalid value encountered in double_scalars
  OF1 = (2 * OP * OR) / (OP + OR) #OF1 (Overall F1 Score) is the harmonic mean of precision and recall.


Train Epoch: 0 [256/3742 (7%)]	Loss: 0.546917 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [320/3742 (8%)]	Loss: 0.514193 	OP: 0.038462	OR: 0.007143	OF1: 0.012048
Train Epoch: 0 [384/3742 (10%)]	Loss: 0.485512 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [448/3742 (12%)]	Loss: 0.460225 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [512/3742 (14%)]	Loss: 0.434832 	OP: 0.032258	OR: 0.007463	OF1: 0.012121
Train Epoch: 0 [576/3742 (15%)]	Loss: 0.408632 	OP: 0.166667	OR: 0.035461	OF1: 0.058480
Train Epoch: 0 [640/3742 (17%)]	Loss: 0.389026 	OP: 0.068966	OR: 0.014599	OF1: 0.024096
Train Epoch: 0 [704/3742 (19%)]	Loss: 0.370935 	OP: 0.096774	OR: 0.021583	OF1: 0.035294
Train Epoch: 0 [768/3742 (20%)]	Loss: 0.346261 	OP: 0.137931	OR: 0.031008	OF1: 0.050633
Train Epoch: 0 [832/3742 (22%)]	Loss: 0.330763 	OP: 0.133333	OR: 0.029851	OF1: 0.048780
Train Epoch: 0 [896/3742 (24%)]	Loss: 0.325184 	OP: 0.151515	OR: 0.035461	OF1: 0.057471
Train Epoch: 0 [960/3742 (25%)]	Loss: 0.295355 	O

<ipython-input-24-ea5e5cd9fa3b>:45: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.2092 
OP: 0.076923
OR: 0.142857
OF1: 0.100000

Train Epoch: 1 [0/3742 (0%)]	Loss: 0.170585 	OP: 0.540000	OR: 0.201493	OF1: 0.293478
Train Epoch: 1 [64/3742 (2%)]	Loss: 0.200910 	OP: 0.500000	OR: 0.200000	OF1: 0.285714
Train Epoch: 1 [128/3742 (3%)]	Loss: 0.187900 	OP: 0.516667	OR: 0.229630	OF1: 0.317949
Train Epoch: 1 [192/3742 (5%)]	Loss: 0.193933 	OP: 0.525424	OR: 0.210884	OF1: 0.300971
Train Epoch: 1 [256/3742 (7%)]	Loss: 0.170857 	OP: 0.587302	OR: 0.282443	OF1: 0.381443
Train Epoch: 1 [320/3742 (8%)]	Loss: 0.186951 	OP: 0.491228	OR: 0.198582	OF1: 0.282828
Train Epoch: 1 [384/3742 (10%)]	Loss: 0.167561 	OP: 0.557377	OR: 0.263566	OF1: 0.357895
Train Epoch: 1 [448/3742 (12%)]	Loss: 0.169442 	OP: 0.491228	OR: 0.227642	OF1: 0.311111
Train Epoch: 1 [512/3742 (14%)]	Loss: 0.201028 	OP: 0.507937	OR: 0.211921	OF1: 0.299065
Train Epoch: 1 [576/3742 (15%)]	Loss: 0.156928 	OP: 0.551724	OR: 0.258065	OF1: 0.351648
Train Epoch: 1 [640/3742 (17%)]	Loss: 0.171310 	OP:

# Model 2: DenseNet201

In [ ]:
# DenseNet201

import torchvision
import torch.nn as nn

model2 = torchvision.models.densenet201(pretrained=True)
num_ftrs = model2.classifier.in_features
model2.classifier = nn.Linear(num_ftrs, len(classes))

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model2.parameters(), lr=learning_rate)

In [ ]:
from statistics import mean
import numpy as np

def train(epoch):
    model2.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model2(data)

        preds = torch.round(output)
            
        #acc_list = []
        #preds = torch.round(output)
        #for i in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for j in range(len(classes)):
        #        if target[i][j] == 1 or preds[i][j] == 1:
        #            denom += 1
        #            if preds[i][j] == target[i][j]:
        #                result+=1
        #    acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target, 'wider')
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model2.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model2(data)
        
        preds = torch.round(output)
        
        #acc_list = []
        #preds = torch.round(output)
        #for n in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for m in range(len(classes)):
        #        if target[n][m] == 1 or preds[n][m] == 1:
        #            denom += 1
        #            if preds[n][m] == target[n][m]:
        #                result+=1
        #    acc_list.append(result/denom)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target, 'wider')

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP, OR, OF1))

for epoch in range(0, epochs):
    train(epoch)
    test()

# Model 3: ResNet18

In [ ]:
# Resnet152 model
from torchvision import models

class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(2048, len(classes))

    def forward(self, x):
        x = self.resnet(x)
        return x

In [ ]:
model3 = ResNet18()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model3.parameters(), lr=learning_rate)

In [ ]:
from statistics import mean
import numpy as np

def train(epoch):
    model3.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model3(data)

        preds = torch.round(output)
            
        #acc_list = []
        #preds = torch.round(output)
        #for i in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for j in range(len(classes)):
        #        if target[i][j] == 1 or preds[i][j] == 1:
        #            denom += 1
        #            if preds[i][j] == target[i][j]:
        #                result+=1
        #    acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target, 'voc07')
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model3.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model3(data)
        
        preds = torch.round(output)
        
        #acc_list = []
        #preds = torch.round(output)
        #for n in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for m in range(len(classes)):
        #        if target[n][m] == 1 or preds[n][m] == 1:
        #            denom += 1
        #            if preds[n][m] == target[n][m]:
        #                result+=1
        #    acc_list.append(result/denom)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target, 'voc07')

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP, OR, OF1))

for epoch in range(0, epochs):
    train(epoch)
    test()

# VGG (frozen)

In [ ]:
class MyCNN3(nn.Module):
    def __init__(self):
        super().__init__()
        model_vgg11 = models.vgg11(weights=models.VGG11_Weights.DEFAULT)
        
#         child_counter = 0
#         for child in model_vgg11.children():
#             if child_counter < 1:
#                 for param in child.parameters():
#                     param.requires_grad = False
#                     child_counter += 1
#             else:
#                 child_counter += 1
        
        self.model_pre = nn.Sequential(*list(model_vgg11.children())[:-1])
        fc_feat = model_vgg11.classifier[0].in_features
        self.model_post = nn.Sequential(
            nn.Linear(fc_feat, 4096, bias = True),
            nn.ReLU(inplace=True),
            nn.Dropout(.5, inplace = False),
            nn.Linear(4096, 4096, bias = True),
            nn.ReLU(inplace=True),
            nn.Dropout(.5, inplace = False),
        )
        self.fc = nn.Linear(4096, len(classes))
    
    def forward(self, X):
        X = self.model_pre(X)
        X = torch.flatten(X,1)
        X = self.model_post(X)
        return self.fc(X)

In [ ]:
model4 = MyCNN3()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model4.parameters(), lr=learning_rate)

In [ ]:
from statistics import mean
import numpy as np

def train(epoch):
    model4.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model4(data)

        preds = torch.round(output)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model4.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model4(data)
        
        preds = torch.round(output)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP, OR, OF1))

for epoch in range(0, epochs):
    train(epoch)
    test()

# VGG (Unfrozen)

In [ ]:
class MyCNN4(nn.Module):
    def __init__(self):
        super().__init__()
        model_vgg11 = models.vgg11(weights=models.VGG11_Weights.DEFAULT)
        
        child_counter = 0
        for child in model_vgg11.children():
            if child_counter < 1:
                for param in child.parameters():
                    param.requires_grad = False
                    child_counter += 1
            else:
                child_counter += 1
        
        self.model_pre = nn.Sequential(*list(model_vgg11.children())[:-1])
        fc_feat = model_vgg11.classifier[0].in_features
        self.model_post = nn.Sequential(
            nn.Linear(fc_feat, 4096, bias = True),
            nn.ReLU(inplace=True),
            nn.Dropout(.5, inplace = False),
            nn.Linear(4096, 4096, bias = True),
            nn.ReLU(inplace=True),
            nn.Dropout(.5, inplace = False),
        )
        self.fc = nn.Linear(4096, len(classes))
    
    def forward(self, X):
        X = self.model_pre(X)
        X = torch.flatten(X,1)
        X = self.model_post(X)
        return self.fc(X)

In [ ]:
model5 = MyCNN4()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model5.parameters(), lr=learning_rate)

In [ ]:
from statistics import mean
import numpy as np

def train(epoch):
    model5.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model5(data)

        preds = torch.round(output)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model5.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model5(data)
        
        preds = torch.round(output)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP, OR, OF1))

for epoch in range(0, epochs):
    train(epoch)
    test()